In [ ]:
## Imports and constants
import os
import sys

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

from lstm_preprocessing import lstm_preprocessing

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42) 
np.random.seed(42)


MAX_SEQ_LEN = 200
BATCH_SIZE = 32

In [ ]:
raw_dataset = pd.read_csv(os.getcwd() + '\\airlines_reviews.csv')
raw_dataset.head()

In [ ]:
## Preprocessing
tokenized_dataset = zs_preprocessing(dataset=raw_dataset.sample(1000, ignore_index=True)) ##TODO: Remove sampling
tokenized_dataset.head()

In [ ]:
## Dataset class
    ## Uses preprocessing method above
    ## __getitem__ returns (preprocessed) text and its corresponding label

class ReviewsDataset(Dataset):
    def __init__(self, reviews, labels):
        self.reviews = reviews
        self.labels = labels

    
    def __len__(self):
        return len(self.labels)


    def __getitem__(self, idx):
        """
        Returns a tuple of a preprocessed review and the corresponding label. If the
        vocabulary is enabled, returns a numerical representation of the review.

        Args:
            idx: a single index
        Returns: a (review, label) tuple
        """
        ret_review = self.reviews[idx]
        ret_label = self.labels[idx]
        
        return ret_review, ret_label

In [ ]:
## Testing the Dataset class
temp_dataset = tokenized_dataset[:10]
test_dataset = ReviewsDataset(reviews=temp_dataset['Tokenized_Reviews'], labels=temp_dataset['Sentiment'])
review, label = test_dataset.__getitem__(3)
print(review)
print(label)

In [ ]:
## Split dataset
train_set, val_set, test_set = torch.utils.data.random_split(
    tokenized_dataset, [0.8, 0.1, 0.1], generator=torch.Generator()
)

In [ ]:
## Create DataLoaders

In [ ]:
## LSTM model
    ## Embedding layer
    ## LSTM (for thought vector)
    ## Linear layer (for logit score)
    ## Activation (for P of +ve sentiment)

In [ ]:
## Test forward pass

In [ ]:
## Evaluation function

In [ ]:
## Training function / Optimization loop

In [ ]:
## Hyperparameter tuning
    ## Dropout probability